In [0]:
# dbutils.library.restartPython()

# import sys
# print(sys.version)

In [0]:
# import shutil
# shutil.rmtree('/local_disk0/repl_tmp_data')

In [0]:
import mlflow
from mlflow.models import Model
from mlflow.tracking import MlflowClient

client = MlflowClient(registry_uri="databricks")

name = 'aisdk_model_state_identifier'

versions = client.search_model_versions(f"name='{name}'")
latest_version = max(versions, key=lambda v: int(v.version))
assert latest_version, f"No versions found for model {name}"
mv = latest_version


model_uri = f"models:/{name}/{latest_version.version}"
model_info = Model.load(model_uri)
signature = model_info.signature
print(f"Model signature: {signature}")

model_folder = client.download_artifacts(run_id=mv.run_id, path="model")  # no dst_path
print("Downloaded to:", model_folder)

print(mv)
version = str(mv.version)
print(f'version: {version}')

In [0]:
# import os
# from pathlib import Path

# # print(f'model_folder: {model_folder}')

# def list_files(directory):
#     for root, dirs, files in os.walk(directory):
#         for name in files:
#             print(os.path.join(root, name))

# # list_files(Path(model_folder).parent.parent)
# list_files('/local_disk0/repl_tmp_data')
# # list_files(model_folder + '/package')



In [0]:
from pathlib import Path
import os

current_dir = Path(os.getcwd())
print(f'current_dir: {current_dir}')

src_package_dir = current_dir / 'package'
print(f'src_package_dir: {src_package_dir}')

print(f'model_folder: {model_folder}')
tmp_model_folder = Path(model_folder).parent.resolve()
print(f'tmp_model_folder: {tmp_model_folder}')

model_file = 'model/model.pkl'
print(f'model_file: {model_file}')

mlops_folder = (current_dir / ".." / "..").resolve()
print(f'mlops_folder: {mlops_folder}')

pipeline_folder_path = model_folder + '/package'
print(f'pipeline_folder_path: {pipeline_folder_path}')



In [0]:
from simaticai import deployment
from simaticai.telemetry.consent import allow_telemetry_cli
import uuid
from pathlib import Path

from simaticai.payloads import PipelineVariable
from simaticai.payloads import Connection, ConnectionTypeAndPayloadFormat as CAPT

allow_telemetry_cli()

component_description = 'component description'
pipeline_description = 'pipeline description'
python_version='3.12.3'

print(f'pipeline_name: {name}')
print(f'version: {version}')

pipeline_file = pipeline_folder_path + f'/{name}_{version}.zip'
print(f'pipeline_file: {pipeline_file}')


package_id = str(uuid.uuid4())

component = deployment.PythonComponent(
        name="inference",
        desc=component_description,
        version='0.0.1',
        python_version=python_version,
    )

pipeline = deployment.Pipeline.from_components(
    [component], name=name, desc=pipeline_description
)

connection_databus_s7 = Connection(name="databusS7", cptype=CAPT.Databus_SIMATICv1)

input_ph1 = PipelineVariable(
    componentName=component.name,
    variableName="ph1",
    tagName="ph1",
    # type="Double",
    connection=connection_databus_s7,
)
input_ph1.type = "Double"

input_ph2 = PipelineVariable(
    componentName=component.name,
    variableName="ph2",
    tagName="ph2",
    # type="Double",
    connection=connection_databus_s7,
)
input_ph2.type = "Double"

input_ph3 = PipelineVariable(
    componentName=component.name,
    variableName="ph3",
    tagName="ph3",
    # type="Double",
    connection=connection_databus_s7,
)
input_ph3.type = "Double"

output_predicition = PipelineVariable(
    componentName=component.name,
    variableName="prediction",
    tagName="prediction",
    # type="Integer",
    connection=connection_databus_s7,
)
output_predicition.type = "Integer"

print(f'component.inputs: {component.inputs}')


pipeline.inputs.append(input_ph1)
pipeline.inputs.append(input_ph2)
pipeline.inputs.append(input_ph3)
pipeline.outputs.append(output_predicition)

component.add_input(input_ph1.variableName, input_ph1.type)
component.add_input(input_ph2.variableName, input_ph2.type)
component.add_input(input_ph3.variableName, input_ph3.type)
component.add_output(output_predicition.variableName, output_predicition.type)




print(f'src_package_dir: {src_package_dir}')
for root, dirs, files in os.walk(src_package_dir):
    for file in files:
        print(os.path.join(root, file))

component.add_resources(
        src_package_dir,
        ["entrypoint.py"],
)
component.set_entrypoint("entrypoint.py")

component.add_resources(tmp_model_folder, model_file)

# component.add_input("ph1", "Double", "Measured energy consumption on phase 1")
# component.add_input("ph2", "Double", "Measured energy consumption on phase 2")
# component.add_input("ph3", "Double", "Measured energy consumption on phase 3")

# component.add_output(
#     "prediction", "Integer", "Predicted cluster of the datapoint (0, 1 or 2)"
# )
component.add_metric("model_input_min")
component.add_metric("model_input_max")
component.add_metric("model_input_mean")

component.add_resources(
    mlops_folder,
    [
        "state_identifier/src/si",
    ],
)

requirements_path = model_folder + "/requirements.txt"
component.set_requirements(requirements_path)


pipeline.add_parameter("step_size", 300, "Integer")
pipeline.set_timeshifting_periodicity(250)


# Ensure a 'package' subdirectory exists under the downloaded model folder
package_dir = Path(pipeline_folder_path)
package_dir.mkdir(parents=True, exist_ok=True)



pipeline_package_path = pipeline.export(
    pipeline_folder_path,
    version=version,
    package_id=package_id,
)



In [0]:
print(mv)

In [0]:
import mlflow
from mlflow.pyfunc import PythonModel

print(f'pipeline_package_path: {pipeline_package_path}')

class ZipBundle(PythonModel):
    def load_context(self, context):
        # The ZIP is available at context.artifacts["bundle"]
        self.bundle_path = context.artifacts["bundle"]
    def predict(self, context, model_input):
        # No-op: return metadata or echo input—your choice
        return {"bundle_path": self.bundle_path}

mlflow.set_registry_uri("databricks")
registered_name = f'{name}-edge'   # <catalog>.<schema>.<model>

with mlflow.start_run():
    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=ZipBundle(),
        artifacts={"bundle": str(pipeline_package_path)},
        registered_model_name=registered_name,
        pip_requirements=requirements_path,
        signature=signature
    )